In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt


In [2]:
se = pd.read_csv("SE test.csv")
om = pd.read_csv("OM test.csv")

In [3]:
se

,AdditionalId,CallPut,Exchange,Expiration,MarketSize,OptionId,Price,Side,Size,Strike,TimeStamp,UnderlyingSymbol
0,0,P,ExP,2022-01-21,15,163577762,152.95,S,12,430.0,2019-06-17 09:30:00.000000,T0
1,1,P,ExP,2019-11-08,8,160689960,9.05,B,6,149.0,2019-06-17 09:30:00.612719,M0
2,2,C,ExA,2020-09-30,547,161165932,24.85,S,454,292.0,2019-06-17 09:30:01.392312,S0
3,3,C,ExP,2019-11-01,1889,159364416,2.03,S,198,7.0,2019-06-17 09:30:01.806161,G0
4,4,C,ExC,2020-12-18,405,146069175,30.84,B,248,287.0,2019-06-17 09:30:02.526837,S0
...,...,...,...,...,...,...,...,...,...,...,...,...
23398,23398,C,ExP,2019-11-29,1524,161875005,3.90,B,851,8.0,2019-06-17 15:59:57.387185,U1
23399,23399,P,ExP,2019-11-15,30,163073417,2.15,B,28,87.0,2019-06-17 15:59:57.897781,S1
23400,23400,C,ExA,2020-04-17,84,157782232,16.55,S,51,31.0,2019-06-17 15:59:58.580998,M1
23401,23401,C,ExP,2019-11-29,6861,161851247,0.60,B,5957,8.0,2019-06-17 15:59:58.870389,D1


In [4]:
om

,AdditionalId,OptionId,OrderEventType,OrderId,TimeStamp
0,0,163577762,NEW,1,2019-06-17 09:30:00.000024
1,0,163577762,CANCEL,1,2019-06-17 09:30:00.001022
2,1,160689960,NEW,2,2019-06-17 09:30:00.612734
3,1,160689960,CANCEL,2,2019-06-17 09:30:00.613724
4,2,161165932,NEW,3,2019-06-17 09:30:01.392333
...,...,...,...,...,...
43321,23400,157782232,NEW,19922,2019-06-17 15:59:58.581020
43322,23400,157782232,CANCEL,19922,2019-06-17 15:59:58.582634
43323,23401,161851247,REJECT,0,2019-06-17 15:59:58.870411
43324,23402,161535283,NEW,19923,2019-06-17 15:59:59.267899


### Q1

In [6]:
def om_checker(om_out, om_in):
    import pandas as pd

    out_sub = om_out.groupby(["AdditionalId","OptionId"], as_index=False).first()[["AdditionalId","OptionId","OrderId"]]
    in_sub  = om_in.groupby(["AdditionalId","OptionId"], as_index=False).first()[["AdditionalId","OptionId","OrderId"]]

    merged = pd.merge(out_sub, in_sub,
                      on=["AdditionalId","OptionId"],
                      how="outer",
                      suffixes=("_out","_in"))

    # NaN-safe inequality check
    def neq(a, b):
        return ((a != b) & ~(a.isna() & b.isna())) | (a.isna() ^ b.isna())

    mask = (merged["OrderId_out"] != merged["OrderId_in"])
    diff = merged.loc[mask].copy()
    return diff


def om_remover(om_out, om_in, unmatched):
    import pandas as pd

    bad_pairs = unmatched[["AdditionalId","OptionId"]].dropna()

    om_out_clean = om_out.merge(bad_pairs, on=["AdditionalId","OptionId"], how="left", indicator=True)
    om_out_clean = om_out_clean[om_out_clean["_merge"] == "left_only"].drop(columns="_merge")

    om_in_clean = om_in.merge(bad_pairs, on=["AdditionalId","OptionId"], how="left", indicator=True)
    om_in_clean = om_in_clean[om_in_clean["_merge"] == "left_only"].drop(columns="_merge")

    return om_out_clean, om_in_clean


In [7]:
om_out = om[om["OrderEventType"]=="NEW"].sort_values(["AdditionalId","TimeStamp"])
om_in = om[om["OrderEventType"].isin(["FILL", "CANCEL"])]
unmatched = om_checker(om_out, om_in)
unmatched

,AdditionalId,OptionId,OrderId_out,OrderId_in
2291,2687,155120457,2292.0,NaN
2292,2688,155120457,NaN,2292.0
4517,5298,159565037,4517.0,NaN
4518,5299,159565037,NaN,4517.0
6318,7437,161874991,6317.0,NaN
6320,7438,161874991,NaN,6317.0
7857,9224,157534926,7855.0,NaN
7858,9225,157534926,NaN,7855.0
19119,22454,161870792,19116.0,NaN
19121,22455,161870792,NaN,19116.0


In [8]:
om_out, om_in = om_remover(om_out, om_in, unmatched)
len(om_out) == len(om_in)

True

In [9]:
def cal_latency(om_out, om_in, se):
    out_ts = om_out.set_index(["AdditionalId", "OptionId"])["TimeStamp"]
    in_ts  = om_in.set_index(["AdditionalId", "OptionId"])["TimeStamp"]
    latency = (pd.to_datetime(in_ts) - pd.to_datetime(out_ts)).dt.total_seconds()
    exch = se.set_index(["AdditionalId","OptionId"])["Exchange"]
    lat_df = pd.DataFrame({"latency(s)": latency}).join(exch)
    return lat_df

In [10]:
om_in

,AdditionalId,OptionId,OrderEventType,OrderId,TimeStamp
0,0,163577762,CANCEL,1,2019-06-17 09:30:00.001022
1,1,160689960,CANCEL,2,2019-06-17 09:30:00.613724
2,2,161165932,CANCEL,3,2019-06-17 09:30:01.393939
3,3,159364416,FILL,4,2019-06-17 09:30:01.808489
4,4,146069175,CANCEL,5,2019-06-17 09:30:02.526913
...,...,...,...,...,...
19918,23397,115439028,CANCEL,19919,2019-06-17 15:59:55.760877
19919,23398,161875005,FILL,19920,2019-06-17 15:59:57.388218
19920,23399,163073417,FILL,19921,2019-06-17 15:59:57.898790
19921,23400,157782232,CANCEL,19922,2019-06-17 15:59:58.582634


In [11]:
latency_df = cal_latency(om_out, om_in, se)
exch_stats = (
        latency_df.groupby("Exchange")["latency(s)"]
              .agg(avg_latency="mean", min_latency="min", max_latency="max", count="count")
              .reset_index())

In [12]:
exch_stats

,Exchange,avg_latency,min_latency,max_latency,count
0,ExA,0.001599,0.001576,0.001622,5042
1,ExC,0.000059,0.000050,0.000068,5064
2,ExI,0.000059,0.000050,0.000068,5000
3,ExP,0.000998,0.000976,0.001022,4811


### Q2

In [14]:
se_clean = se.dropna(how="any").reset_index(drop=True)
len(se_clean) == len(se)
se = pd.read_csv("SE test.csv")

In [15]:
se

,AdditionalId,CallPut,Exchange,Expiration,MarketSize,OptionId,Price,Side,Size,Strike,TimeStamp,UnderlyingSymbol
0,0,P,ExP,2022-01-21,15,163577762,152.95,S,12,430.0,2019-06-17 09:30:00.000000,T0
1,1,P,ExP,2019-11-08,8,160689960,9.05,B,6,149.0,2019-06-17 09:30:00.612719,M0
2,2,C,ExA,2020-09-30,547,161165932,24.85,S,454,292.0,2019-06-17 09:30:01.392312,S0
3,3,C,ExP,2019-11-01,1889,159364416,2.03,S,198,7.0,2019-06-17 09:30:01.806161,G0
4,4,C,ExC,2020-12-18,405,146069175,30.84,B,248,287.0,2019-06-17 09:30:02.526837,S0
...,...,...,...,...,...,...,...,...,...,...,...,...
23398,23398,C,ExP,2019-11-29,1524,161875005,3.90,B,851,8.0,2019-06-17 15:59:57.387185,U1
23399,23399,P,ExP,2019-11-15,30,163073417,2.15,B,28,87.0,2019-06-17 15:59:57.897781,S1
23400,23400,C,ExA,2020-04-17,84,157782232,16.55,S,51,31.0,2019-06-17 15:59:58.580998,M1
23401,23401,C,ExP,2019-11-29,6861,161851247,0.60,B,5957,8.0,2019-06-17 15:59:58.870389,D1


In [16]:
se["book_ratio"] = se["Size"]/se["MarketSize"]
se["dollar_qty"] = se["Price"]*se["Size"]
dt_days = (pd.to_datetime(se["Expiration"]) - pd.to_datetime(se["TimeStamp"])).dt.total_seconds() / 86400.0
se["days_to_expiry"] = dt_days.clip(lower=0)
se["CallPut_Side"]  = (se["CallPut"].astype(str) + "_" + se["Side"].astype(str))
se["Price/Strike"] = se["Price"]/se["Strike"]
se["time_value_intensity"] = (se["Price"] / se["Strike"]) / (se["days_to_expiry"] + 1e-6)

ts = pd.to_datetime(se["TimeStamp"])
sec = ts.dt.hour * 3600 + ts.dt.minute * 60 + ts.dt.second + ts.dt.microsecond/1e6 + 1e-3
open_s, close_s = 9*3600 + 30*60, 16*3600
mid = (open_s + close_s) / 2.0

dist   = np.where(sec <= mid, sec - open_s, close_s - sec)
signed = np.where(sec <= mid, -dist, dist)

scale = 1 / ((close_s - open_s) / 2.0)
se["market_time"] = 1/np.tanh(signed*scale)

In [17]:
om_y = om[om["OrderEventType"].isin(["NEW", "REJECT"])].copy()

y_map = {"NEW": 1, "REJECT": 0}
om_y["Y"] = om_y["OrderEventType"].map(y_map)

se_labeled = se.merge(
    om_y[["AdditionalId","OptionId","OrderEventType","Y"]],
    on=["AdditionalId","OptionId"],
    how="left"
)

missing = se_labeled["Y"].isna().sum()
se_labeled=se_labeled.drop(columns=[c for c in ["AdditionalId","OptionId","Expiration","OrderEventType","CallPut", "Side", "UnderlyingSymbol"] if c in se_labeled.columns])

In [18]:
se_labeled

,Exchange,MarketSize,Price,Size,Strike,TimeStamp,book_ratio,dollar_qty,days_to_expiry,CallPut_Side,Price/Strike,time_value_intensity,market_time,Y
0,ExP,15,152.95,12,430.0,2019-06-17 09:30:00.000000,0.800000,1835.40,948.604167,P_S,0.355698,0.000375,-1.170000e+07,1
1,ExP,8,9.05,6,149.0,2019-06-17 09:30:00.612719,0.750000,54.30,143.604160,P_B,0.060738,0.000423,-1.906410e+04,1
2,ExA,547,24.85,454,292.0,2019-06-17 09:30:01.392312,0.829982,11281.90,470.604151,C_S,0.085103,0.000181,-8.397258e+03,1
3,ExP,1889,2.03,198,7.0,2019-06-17 09:30:01.806161,0.104817,401.94,136.604146,C_S,0.290000,0.002123,-6.474243e+03,1
4,ExC,405,30.84,248,287.0,2019-06-17 09:30:02.526837,0.612346,7648.32,549.604137,C_B,0.107456,0.000196,-4.628463e+03,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23398,ExP,1524,3.90,851,8.0,2019-06-17 15:59:57.387185,0.558399,3318.90,164.333364,C_B,0.487500,0.002967,4.479644e+03,1
23399,ExP,30,2.15,28,87.0,2019-06-17 15:59:57.897781,0.933333,60.20,150.333358,P_B,0.024713,0.000164,5.568196e+03,1
23400,ExA,84,16.55,51,31.0,2019-06-17 15:59:58.580998,0.607143,844.05,304.333350,C_S,0.533871,0.001754,8.251046e+03,1
23401,ExP,6861,0.60,5957,8.0,2019-06-17 15:59:58.870389,0.868241,3574.20,164.333346,C_B,0.075000,0.000456,1.036673e+04,0


In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

def run_lgbm_cv(df: pd.DataFrame, n_splits=4, seed=42):
    df = df.copy()

    target_col = "Y"
    drop_cols = [c for c in ["Y", "TimeStamp"] if c in df.columns] 
    feat_cols = [c for c in df.columns if c not in drop_cols]
    cat_cols = [c for c in ["Exchange", "CallPut_Side",'CallPut']
                if c in feat_cols]
    for c in cat_cols:
        df[c] = df[c].astype("category")

    X = df[feat_cols]
    y = df[target_col].astype(int)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    oof_pred = np.zeros(len(df), dtype=float)
    oof_cls  = np.zeros(len(df), dtype=int)
    models = []
    imp_list = []

    aucs, accs, f1s = [], [], []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        model = lgb.LGBMClassifier(
            n_estimators=500,
            learning_rate=0.002,
            num_leaves=31,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            min_child_samples=30,
            objective="binary",
            n_jobs=-1,
        )

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="auc",
            categorical_feature=cat_cols,
            callbacks=[lgb.early_stopping(100, verbose=30)]
        )

        p = model.predict_proba(X_va)[:, 1]
        c = (p >= 0.5).astype(int)

        
        try:
            auc = roc_auc_score(y_va, p)
        except ValueError:
            auc = np.nan
        acc = accuracy_score(y_va, c)
        f1  = f1_score(y_va, c, zero_division=0)

        aucs.append(auc); accs.append(acc); f1s.append(f1)

        oof_pred[va_idx] = p
        oof_cls[va_idx] = c
        models.append(model)

        # feature importance (gain)
        imp = pd.DataFrame({
            "feature": feat_cols,
            f"gain_fold{fold}": model.booster_.feature_importance(importance_type="gain")
        })
        imp_list.append(imp)

        print(f"[fold {fold}] AUC={auc:.4f} ACC={acc:.4f} F1={f1:.4f}  best_iter={model.best_iteration_}")


    print("\n=== CV summary ===")
    print(f"AUC: {np.nanmean(aucs):.4f} ± {np.nanstd(aucs):.4f}")
    print(f"ACC: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"F1 : {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")


    try:
        overall_auc = roc_auc_score(y, oof_pred)
        print(f"OOF AUC: {overall_auc:.4f}")
    except ValueError:
        pass

    feat_imp = imp_list[0][["feature"]].copy()
    for imp in imp_list:
        feat_imp = feat_imp.merge(imp, on="feature", how="left")
    gain_cols = [c for c in feat_imp.columns if c.startswith("gain_fold")]
    feat_imp["gain_avg"] = feat_imp[gain_cols].mean(axis=1)
    feat_imp = feat_imp.sort_values("gain_avg", ascending=False).reset_index(drop=True)

    return {
        "models": models,
        "oof_pred": oof_pred,
        "oof_cls": oof_cls,
        "cv_auc": aucs,
        "cv_acc": accs,
        "cv_f1": f1s,
        "feature_importance": feat_imp,
        "features": feat_cols,
        "categoricals": cat_cols,
    }

In [20]:
res = run_lgbm_cv(se_labeled, n_splits=5, seed=42)

[LightGBM] [Info] Number of positive: 15938, number of negative: 2784
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000356 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2560
[LightGBM] [Info] Number of data points in the train set: 18722, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.851298 -> initscore=1.744817
[LightGBM] [Info] Start training from score 1.744817
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [21]:
res

{'models': [LGBMClassifier(colsample_bytree=0.8, learning_rate=0.002, max_depth=5,
                 min_child_samples=30, n_estimators=500, n_jobs=-1,
                 objective='binary', subsample=0.8),
  LGBMClassifier(colsample_bytree=0.8, learning_rate=0.002, max_depth=5,
                 min_child_samples=30, n_estimators=500, n_jobs=-1,
                 objective='binary', subsample=0.8),
  LGBMClassifier(colsample_bytree=0.8, learning_rate=0.002, max_depth=5,
                 min_child_samples=30, n_estimators=500, n_jobs=-1,
                 objective='binary', subsample=0.8),
  LGBMClassifier(colsample_bytree=0.8, learning_rate=0.002, max_depth=5,
                 min_child_samples=30, n_estimators=500, n_jobs=-1,
                 objective='binary', subsample=0.8),
  LGBMClassifier(colsample_bytree=0.8, learning_rate=0.002, max_depth=5,
                 min_child_samples=30, n_estimators=500, n_jobs=-1,
                 objective='binary', subsample=0.8)],
 'oof_pred': array(

In [22]:
res['feature_importance']

,feature,gain_fold1,gain_fold2,gain_fold3,gain_fold4,gain_fold5,gain_avg
0,Strike,562087.744914,24458.490575,564020.412220,563305.707474,578772.053081,458528.881653
1,Price/Strike,154440.892296,13104.709572,149978.663947,155898.457110,162484.930192,127181.530623
2,time_value_intensity,153102.122415,8690.256308,153838.646724,148377.992752,156585.424105,124118.888461
3,Price,123789.202754,9451.761250,116469.141060,123548.317934,125016.490844,99654.982768
4,MarketSize,111742.285379,6093.010742,97400.404469,102501.386922,91983.059889,81944.029480
5,CallPut_Side,87216.402252,6565.288223,100313.448139,99484.757990,93870.846956,77490.148712
6,days_to_expiry,66571.290308,5032.852744,62819.572076,65576.509495,58928.870723,51785.819069
7,dollar_qty,10835.827601,1179.535848,10108.767650,10684.773797,9487.100777,8459.201135
8,Size,4388.907115,1054.251565,3782.609847,3825.590055,5359.840855,3682.239887
9,book_ratio,1512.508015,418.319073,2888.201133,1535.904475,3126.670996,1896.320738


In [ ]:
https://pinnate-juniper-a95.notion.site/Data-Science-2e3eacff7779805daab3f6be180b346b?source=copy_link